In [ ]:
"""
Final classification script
"""

'\nFinal classification script\n'

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
##### GIVE INPUTS HERE ######

snapshot_output_dir_name = '2019-09-07 to 2022-09-06'

#############################

# Defining Directory paths
default_dir = '/content/drive/MyDrive/WheelPros'
output_dir = default_dir + '/output'
mapping_dir = default_dir + '/input/Mapping Data'
snapshot_output_dir = output_dir +'/'+ snapshot_output_dir_name

# Defining Filenames
sales_file = 'Processed Sales data.csv'
material_list_file = 'All Inventory Wheels Only 9.1.2022.xlsx'

In [ ]:
# Reading Sales file
os.chdir(snapshot_output_dir)
sales_df = pd.read_csv(sales_file).astype({'Material':'str','Created on':'datetime64'})

In [ ]:
sales_df.head()

,Material,Plnt,Profit Ctr,Bill.qty in SKU,Net value,Curr.,Created on,City,Cty,Zip Code,...,Zip_optimal_lc,Zip_optimal_dc,Source,Is Optimal DC,Is Optimal LC,Is Other DC,Is Other LC,Is Optimal,distance_source,distance_optimal_lc
0,D51720907057,1003,1003.0,4,884.00,USD,2020-12-01,HOUSTON,US,77081,...,75050,77043,Optimal DC,1,0,0,0,1,0.000000,221.699062
1,M119209065+35,1030,1022.0,4,733.04,USD,2020-12-01,2038 26th St Sw,US,52404,...,75050,60124,Other DC,0,0,1,0,0,656.888607,795.594338
2,XD81189035700,1030,1022.0,1,155.29,USD,2020-12-01,dellwood,US,55110,...,75050,55428,Other DC,0,0,1,0,0,973.631953,871.603001
3,XD82529067300,1030,1030.0,1,205.10,USD,2020-12-01,WARRENTON,US,20186,...,30519,21076,Other DC,0,0,1,0,0,115.677755,531.474911
4,HE182756060306N,1024,1024.0,36,2682.00,USD,2020-12-01,VONORE,US,37885,...,30519,37072,Optimal DC,1,0,0,0,1,0.000000,221.879012


In [ ]:
def perform_classification(dc_sales_df, dc):
  
  # dc_id = int(sales_df[sales_df['Optimal_DC'] == 1028].head(1)['Optimal_DC']) # Fetching the DC id
  
  # Calculating Days since previous sale
  dc_sales_df.sort_values('Created on', inplace=True)
  diff_date = dc_sales_df['Created on'] - dc_sales_df.groupby(['Material'])['Created on'].shift()
  days_since_last_sale = [i.days for i in diff_date]  
  dc_sales_df['Days since previous sale'] = days_since_last_sale

  # Aggregating to material level
  dc_sales_df_grouped = dc_sales_df.groupby(['Material']).agg({'Created on': ['min','max','nunique'],
                                                     'Bill.qty in SKU':['sum','mean','count'],
                                                     'Net value':['sum','mean'],
                                                     'Days since previous sale':['mean', 'std'],
                                                     'Plnt':['nunique'],
                                                     'Optimal_DC':['nunique'],
                                                     'Is Optimal DC':['mean'],
                                                     'Is Optimal LC':['mean'],
                                                     'Is Other DC':['mean'],
                                                     'Is Other LC':['mean'],
                                                     'Is Optimal':['mean']
                                                    }).reset_index()
  total_days_between_sale = dc_sales_df_grouped[('Created on','max')] - dc_sales_df_grouped[('Created on','min')]
  days_since_last_sold = max(dc_sales_df_grouped[('Created on','max')]) - dc_sales_df_grouped[('Created on','max')]

  total_days_between_sale = [i.days for i in total_days_between_sale]
  days_since_last_sold = [i.days for i in days_since_last_sold]

  dc_sales_df_grouped[('Days b/w first and last sale', '')] = total_days_between_sale
  dc_sales_df_grouped[('Days since last sale', '')] = days_since_last_sold

  ## Binning based on percentile performance
  
  # 1. Frequency
  dc_sales_df_grouped[('Category_Frequency','')] = pd.qcut(dc_sales_df_grouped[('Bill.qty in SKU', 'count')], q=[0, 0.75, 1],
                                                labels=["low", "high"])
  
  df_grouped_highFreq = dc_sales_df_grouped[dc_sales_df_grouped[('Category_Frequency', '')] == 'high']
  df_grouped_lowFreq = dc_sales_df_grouped[dc_sales_df_grouped[('Category_Frequency', '')] == 'low']

  # 2. Volume, Price, Variance
  
  # For high frequency SKUs:
  
  df_grouped_highFreq[('Category_Volume','')] = 'NA'
  df_grouped_highFreq[('Category_Price', '')] = 'NA'
  df_grouped_highFreq[('Category_Variance', '')] = 'NA'

  # For low frequency SKUs:
  # Filtering rare sellers
  df_grouped_lowFreq_rare = df_grouped_lowFreq[df_grouped_lowFreq[('Bill.qty in SKU','count')] <= 2]
  df_grouped_lowFreq_rare[('Category_Volume','')] = 'NA'
  df_grouped_lowFreq_rare[('Category_Price', '')] = 'NA'
  df_grouped_lowFreq_rare[('Category_Variance', '')] = 'high'

  # Classifying for remaining
  df_grouped_lowFreq = df_grouped_lowFreq[df_grouped_lowFreq[('Bill.qty in SKU','count')] > 2]

  try:
    df_grouped_lowFreq[('Category_Volume','')] = pd.qcut(df_grouped_lowFreq[('Bill.qty in SKU', 'sum')], q=[0, 0.75, 1],
                                             labels=["low", "high"])
  except:
    print(f'Issue in Volume classification at {dc}')
    df_grouped_lowFreq[('Category_Volume','')] = 'low'

  try:
    df_grouped_lowFreq[('Category_Price','')] = pd.qcut(df_grouped_lowFreq[('Net value', 'mean')], q=[0, 0.75, 1],
                                            labels=["low", "high"])
  except:
    print(f'Issue in Price classification at {dc}')
    df_grouped_lowFreq[('Category_Price','')] = 'low'

  try:
    df_grouped_lowFreq[('Category_Variance','')] = pd.qcut(df_grouped_lowFreq[('Days since previous sale', 'std')], q=[0, 0.25, 1],
                                               labels=["low", "high"])
  except:
    print(f'Issue in Variance classification at {dc}')
    df_grouped_lowFreq[('Category_Variance','')] = 'high'

  # Final classification
  dc_binned_df = pd.concat((df_grouped_highFreq, df_grouped_lowFreq, df_grouped_lowFreq_rare), axis=0)
  
  def assign_classification(material):
    if material['Category_Frequency'] == 'high':
      return 'FMov'

    elif ((material['Category_Volume'] == 'high') and
          (material['Category_Price'] == 'high') and
          (material['Category_Variance'] == 'low')):
      return 'FMov*'
    
    else:
      return 'SMov'
  
  # Assigning class
  dc_binned_df_fil = dc_binned_df[[('Material', ''),('Category_Frequency', ''), ('Category_Volume',''), ('Category_Price',''), ('Category_Variance','')]]
  dc_binned_df_fil.columns = ['Material','Category_Frequency','Category_Volume','Category_Price','Category_Variance']
  
  dc_binned_df_fil['Category'] = dc_binned_df_fil.apply(lambda material: assign_classification(material), axis=1)

  final_classified_df = dc_binned_df_fil[['Material','Category']]
  final_classified_df.columns = ['Material', f'Class at {dc}']

  return final_classified_df

In [ ]:
dc_list = list(sales_df['Optimal_DC'].unique())

In [ ]:
# DC Level Classification
# Performing categorization and joining
final_class_df = pd.DataFrame(data=list(sales_df['Material'].drop_duplicates()), columns=['Material'], dtype='str')
for dc in dc_list:
  df_catgorized = perform_classification(sales_df[sales_df['Optimal_DC'] == dc], dc)
  final_class_df = final_class_df.merge(df_catgorized.astype('str'), on='Material', how='left')

Issue in Volume classification at 1026
Issue in Price classification at 1026
Issue in Variance classification at 1026


In [ ]:
lc_list = list(sales_df['Optimal_LC'].dropna().astype('int32').unique())

In [ ]:
# LC Level Classification
# Performing categorization and joining
for lc in lc_list:
  df_catgorized = perform_classification(sales_df[sales_df['Optimal_LC'] == lc], lc)
  final_class_df = final_class_df.merge(df_catgorized.astype('str'), on='Material', how='left')

In [ ]:
# US-wide classification
df_catgorized = perform_classification(sales_df, 'US agg')
final_class_df = final_class_df.merge(df_catgorized.astype('str'), on='Material', how='left')

In [ ]:
final_class_df = final_class_df.fillna('NotSold')
final_class_df.head()

,Material,Class at 1003,Class at 1015,Class at 1028,Class at 1034,Class at 1024,Class at 1014,Class at 1016,Class at 1021,Class at 1030,...,Class at 1032,Class at 1013,Class at 1007,Class at 1004,Class at 1008,Class at 1035,Class at 1086,Class at 1088,Class at 1085,Class at US agg
0,D51720907057,FMov,FMov,FMov,FMov,FMov,FMov,SMov,FMov,SMov,...,FMov,FMov,FMov,SMov,FMov,SMov,FMov,FMov,FMov,FMov
1,M119209065+35,FMov,FMov,SMov,FMov,FMov,FMov,FMov,FMov,FMov,...,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov
2,XD81189035700,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov,...,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov
3,XD82529067300,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov,...,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov,FMov
4,HE182756060306N,NotSold,NotSold,NotSold,NotSold,FMov,SMov,NotSold,NotSold,NotSold,...,NotSold,NotSold,NotSold,NotSold,NotSold,NotSold,NotSold,FMov,NotSold,FMov


In [ ]:
# Unpivotting classification table
unpivoted_classification_df = final_class_df.drop('Class at US agg', axis=1).melt(id_vars=['Material'], var_name='Center', value_name='Classification')
unpivoted_classification_df['Center id'] = [x.split(' ')[-1] for x in unpivoted_classification_df['Center']]

In [ ]:
## Applying Styling to classification sheet
def highlight_cells(cell):
  if cell == 'FMov':
    color = '#C4D79B'
  elif cell == 'FMov*':
    color = '#D8E4BC'
  elif cell == 'SMov':
    color = '#FCD5B4'
  elif cell == 'NotSold':
    color = '#BFBFBF'
  else:
    color = ''
  return color

styled_df = final_class_df.set_index('Material').style.applymap(lambda x: f'background-color : {highlight_cells(x)}')\
                          .set_properties(**{'color':'black'})

In [ ]:
# Saving final output
os.chdir(snapshot_output_dir)

styled_df.to_excel(f'Material Classification {snapshot_output_dir_name}.xlsx', index=True)

unpivoted_classification_df.to_csv(f'Unpivoted Material Classification.csv', index=False)

In [ ]:
print(f'Total {len(final_class_df)} different SKU\'s sold')

Total 53138 different SKU's sold
